# YouTube Data API v3

In [2]:
import requests
import json

import pandas as pd

# api_key is stored in config.py
import config

### 1 - Youtube Search (query, n)
    Returns list of channel dictionaries
### 2 - Youtube Channel List
    Returns details on a specific channel
#### A - Request channel details (channelId)
    Returns a json dictionary for a specific channel Id
#### B - Run Channel List (list of channelIds)
    Returns a list of dictionaries per channel

Search method iterating using tokens to get more than 50 results. 

Uses requests to grab type=channel, part=snippet, order=?;

Creates an empty list, extends the list with the json's reponse items (50 at a time).

In [43]:
# Search for channels by a specific query
def request_youtube_search_channels(query, n):
    '''Returns a list of n channels that match the query.\n
    Uses /youtube/v3/search'''
    
    # Logic to determine maxResults parameter
    
    # Empty list to store 50 items from each response
    channel_list = list()
    
    nextPageToken = ''

    while len(channel_list)<n:
        # Debugging
        print(len(channel_list), n)
        
        # Logic for determining maxResults
        
        # Create request object
        resp = requests.get(
            'https://www.googleapis.com/youtube/v3/search',
            params=dict(part='snippet', 
                        type='channel',
                        maxResults=20,
                        pageToken=nextPageToken,
                        q=query,
                        key=config.api_key)
        )
        nextPageToken = json.loads(resp.content)['nextPageToken']
        channel_list.extend(json.loads(resp.content)['items'])
        
    return channel_list

#CHANNEL_LIST = request_youtube_search_channels('Corridor',20)

# Extract Channel Ids from the list of dictionaries
#CHANNELID_LIST = [channel['snippet']['channelId'] for channel in CHANNEL_LIST]

0 20


Function to retrieve channel details by channelId. Returns the entire dictionary from the json response object.

In [23]:
# Get channel details and snippet
def request_channel_list(channelId):
    resp = requests.get(
        'https://www.googleapis.com/youtube/v3/channels',
        params=dict(part='contentDetails, snippet, statistics, brandingSettings, topicDetails, status, id, contentOwnerDetails',
                id=channelId,
                key=config.api_key),
    )
    return json.loads(resp.content)

In [31]:
# Returns a list of json dictionaries for a given list of channelIds
# Iteratively runs requests for channel details and retrieves detail items

def run_channel_list(channelId_list):
    
    # Instantiate empty channel details list
    channel_details_list = list()
    
    # Loop over each channelId in channelId_list
    for channelId in channelId_list:
        
        # Extend channel details list with each channels json dictionary
        channel_details_list.extend(request_channel_list(channelId)['items'])
        
    return channel_details_list

#CHANNEL_DETAILS_LIST = run_channel_list(CHANNELID_LIST)

In [ ]:
# Returns a list of dictionaries ready to be loaded into a dataframe
# Transforms a list of json dictionaries into workable dictionaries for dataframe

def extract_channel_details(details_list):
    # Instantite empty channels details list for new dictionary format
    channel_details_list = list()
    
    # Loop over each channel's json dictionary in details_list
    for channel in details_list:
        # Instantiate new details dictionary; in series update the dictionary to include relevant details
        channel_details_dict = {}
        channel_details_dict.update(dict(id=channel['id']))
        channel_details_dict.update(channel['snippet'])
        channel_details_dict.update(channel['contentDetails'])
        channel_details_dict.update(channel['topicDetails'])
        channel_details_dict.update(channel['status'])
        channel_details_dict.update(channel['statistics'])
        channel_details_dict.update(channel['brandingSettings']['channel'])
        
        # Append the added channel's new dictionary format to channel details list
        channel_details_list.append(channel_details_dict)
    return channel_details_list

#extract_channel_details(CHANNEL_DETAILS_LIST)

In [122]:
# Request n number of channels from a youtube search
CHANNEL_LIST = request_youtube_search_channels('Minecraft',20)

# Retrieve Channel Ids from the list of dictionaries
CHANNELID_LIST = [channel['snippet']['channelId'] for channel in CHANNEL_LIST]

# Request details from each channel
CHANNEL_DETAILS_LIST = run_channel_list(CHANNELID_LIST)

# Extract channel details to a list of dictionaries for pandas
CHANNEL_DETAILS_LIST_CLEAN = extract_channel_details(CHANNEL_DETAILS_LIST)

# Insert list of dictionaries into pandas dataframe
df = pd.DataFrame(CHANNEL_DETAILS_LIST_CLEAN)

0 20


In [141]:
'featuredChannelsUrls' in CHANNEL_DETAILS_LIST_CLEAN[1].keys()

True

In [156]:
{channel['id']:'test' if 'featuredChannelsUrls' in channel.keys() else None for channel in CHANNEL_DETAILS_LIST_CLEAN}

{'UCQvWX73GQygcwXOTSf_VDVg': None,
 'UC1sELGmy5jp5fQUugmuYlXQ': 'test',
 'UCuPfkZuwz7kyNjCV5Uwk3ow': 'test',
 'UC1ANc72wQdusmOHwOL1-93g': 'test',
 'UCcoXbmaUfns8Kx4E5YbwIZA': 'test',
 'UCz_yrc3BO9ve7UgDGZn4dig': 'test',
 'UCf0yodVLuyYH2L-5LJ4Jh4Q': 'test',
 'UCcgjtlfEV_icht1pfQVvUag': 'test',
 'UCR4xGfaPsPctQvKRctFSI2g': 'test',
 'UChYLyN1EcREWIVsr5Uac2Ag': 'test',
 'UCNodmx1ERIjKqvcJLtdzH5Q': None,
 'UCnzTfF2PwXuiwxqERpZexCA': 'test',
 'UCTLGADoU3wF_iUXGXxv2QAw': 'test',
 'UCG5DCJNmyhyCXy7l7fjdR0A': 'test',
 'UC9kilajrLqUbv4cWXwgvcGQ': 'test',
 'UC0kdsLBisFJ2gRkRRwkw8cw': None,
 'UChdf3rY-t0bisRJFBWw2Jxg': 'test',
 'UCWzMOBNLdpwMExNH4BCniWQ': 'test',
 'UCjZRTyx4-If-nVCTHFJo0NA': 'test',
 'UC_7-v7DNaJwpx-4e1jRzcbA': 'test'}

In [158]:
{channel['id']:channel['featuredChannelsUrls'] if 'featuredChannelsUrls' in channel.keys() else None for channel in CHANNEL_DETAILS_LIST_CLEAN}

{'UCQvWX73GQygcwXOTSf_VDVg': None,
 'UC1sELGmy5jp5fQUugmuYlXQ': ['UCmYEOuRIVGSO5PlExUYHqYQ'],
 'UCuPfkZuwz7kyNjCV5Uwk3ow': ['UC4f1zAG2BTkfOQV4_nFbpBQ',
  'UCoVSaXJtW0a8E1iCD6Nm7FA',
  'UCcoXbmaUfns8Kx4E5YbwIZA',
  'UCy2TySx_6AaPvzMRwYMvHxw'],
 'UC1ANc72wQdusmOHwOL1-93g': ['UCO24w9B_-i-qZmss01_N6AA',
  'UC447C0VgAnGRXZFGY51KtKA'],
 'UCcoXbmaUfns8Kx4E5YbwIZA': ['UCI8jXahv2zK6QC2-ea0s20Q',
  'UC4f1zAG2BTkfOQV4_nFbpBQ',
  'UCuPfkZuwz7kyNjCV5Uwk3ow',
  'UCoVSaXJtW0a8E1iCD6Nm7FA',
  'UCy2TySx_6AaPvzMRwYMvHxw'],
 'UCz_yrc3BO9ve7UgDGZn4dig': ['UCfEZ73970aS121NTvh8INGQ',
  'UCArxCs_uuTtbdm7TIFfLOwg',
  'UCuzOJ2nDjndekST3vtiIj9w',
  'UCr9EfNfs0ugh-LSCSJTw8-A',
  'UCf-2ttQNiHRQv3hhP85kYQA',
  'UC_MHYsPj4q0jVPL9Zh3iI9w'],
 'UCf0yodVLuyYH2L-5LJ4Jh4Q': ['UCRWbqpy_JUhatPTOmk0M9Kg',
  'UC4-Hml9a0PeJ85-MpWo6ppA'],
 'UCcgjtlfEV_icht1pfQVvUag': ['UCogmcle_FtrvKKr5UPOLfbA'],
 'UCR4xGfaPsPctQvKRctFSI2g': ['UCXZDQauLr7CrmLsju_6I_qQ',
  'UCv3LcCboNFN6T7cZZ1W0qbw',
  'UCb_FjUA7LAciZCiWhy3qTmQ',
  'UCenY_uRTbm

In [123]:
df.columns

Index(['id', 'title', 'description', 'publishedAt', 'thumbnails', 'localized',
       'relatedPlaylists', 'privacyStatus', 'isLinked', 'longUploadsStatus',
       'viewCount', 'commentCount', 'subscriberCount', 'hiddenSubscriberCount',
       'videoCount', 'showBrowseView', 'profileColor', 'customUrl', 'country',
       'topicIds', 'topicCategories', 'madeForKids', 'keywords', 'defaultTab',
       'trackingAnalyticsAccountId', 'moderateComments', 'showRelatedChannels',
       'featuredChannelsTitle', 'featuredChannelsUrls', 'unsubscribedTrailer',
       'defaultLanguage'],
      dtype='object')

In [125]:
features = ['id','title','description','customUrl','publishedAt','country','isLinked', 'viewCount', 'commentCount', 'subscriberCount',
           'hiddenSubscriberCount','keywords','showRelatedChannels','showBrowseView']
df[features]

,id,title,description,customUrl,publishedAt,country,isLinked,viewCount,commentCount,subscriberCount,hiddenSubscriberCount,keywords,showRelatedChannels,showBrowseView
0,UCQvWX73GQygcwXOTSf_VDVg,Minecraft - Topic,Minecraft is a sandbox video game developed by...,NaN,2013-12-20T22:46:47Z,NaN,True,0,0,14200000,False,NaN,NaN,True
1,UC1sELGmy5jp5fQUugmuYlXQ,Minecraft,This is the official YouTube channel of Minecr...,minecraft,2010-12-21T21:36:46Z,SE,True,967291170,0,5180000,False,Minecraft Mojang MINECON Gaming Microsoft Game...,True,True
2,UCuPfkZuwz7kyNjCV5Uwk3ow,Cannibal Crab | Minecraft,Minecraft has a new Evil Villian in town and t...,NaN,2016-04-13T02:17:45Z,US,True,87180142,0,394000,False,"minecraft ""minecraft roleplay"" challenge ""mine...",True,True
3,UC1ANc72wQdusmOHwOL1-93g,BlueNerd Minecraft,Welcome to BlueNerd Minecraft! This is gaming ...,bluenerdminecraft,2017-11-01T13:48:04Z,GB,True,10203238,0,94000,False,"minecraft ""minecraft how to build"" ""minecraft ...",True,True
4,UCcoXbmaUfns8Kx4E5YbwIZA,Kraken Kid | Minecraft,Minecraft has a new Evil Villian in town and t...,krakenkidminecraft,2015-12-13T16:22:58Z,US,True,265639357,0,782000,False,"minecraft ""the atlantic craft"" games ""kraken k...",True,True
5,UCz_yrc3BO9ve7UgDGZn4dig,MC Naveed - Minecraft & Roblox and MORE!,Hello everyone and welcome back to my Minecraf...,mcnaveed,2013-03-15T13:43:46Z,GB,True,1418375490,0,2390000,False,minecraft,True,True
6,UCf0yodVLuyYH2L-5LJ4Jh4Q,CubicKat - Minecraft Animations,Hello! If you enjoy my content please subscrib...,cubickatminecraftanimations,2018-06-25T03:19:25Z,US,True,20342750,0,127000,False,"""monster school"" minecraft ""monster school ani...",True,True
7,UCcgjtlfEV_icht1pfQVvUag,Latenci - Minecraft,Latenci's second channel.,latenci,2013-07-29T20:10:36Z,US,True,16679709,0,156000,False,Latenci,True,True
8,UCR4xGfaPsPctQvKRctFSI2g,A1MOSTADDICTED MINECRAFT,A1mostaddicted here Minecraft building tutori...,a1mostaddictedminecraft,2014-12-12T19:47:26Z,GB,True,71896158,0,366000,False,MinecraftBuilding minecrafttutorial minecrafth...,True,True
9,UChYLyN1EcREWIVsr5Uac2Ag,Sharky Minecraft Adventures - The Little Club,Hey guys it's Sharky & Scuba Steve from The Li...,NaN,2015-07-06T19:16:07Z,IE,True,493325938,0,1140000,False,"""sharky and scuba steve"" sharky ""scuba steve"" ...",True,True
